# LOGS
Be able ot write logs in order to take back missing data if tath's the case 

In [7]:
import sys
import os
folders_names = ["utilities", "matches", "tournament", "webdrivers"]
base_path = os.path.dirname(os.getcwd())

parent_dir = os.path.join(base_path, folders_names[0], "logger")

# Add the 'a' directory to Python's list of paths to look in
sys.path.append(os.path.join(parent_dir, 'a'))

In [4]:
parent_dir

'c:\\Users\\beAtz\\OneDrive\\Bureau\\WTA_PROJECT\\utilities'

In [ ]:
import os 
os.path.join(base_path, 'utilities')

In [8]:
from utilities.logger import Logger
import datetime

ModuleNotFoundError: No module named 'utilities'

In [ ]:
import os 
os.getcwd()

In [ ]:
run_id = datetime.datetime.now().strftime('%Y%m%d_%H%M')
run_id

In [ ]:
logger = Logger(run_id)
logger.initialization()

# IMPORT LIBRARIES FROM THE DIFFERENT FUNCTIONS

In [ ]:
from tournaments.tournaments_wta import *
import time
import os
import functools

In [ ]:
#global variables
cp_count = os.cpu_count()

## I.Focusing on tournaments for a year

In [ ]:
year = "2023"
logger.log_info(f"Retrieving tournaments played during {year}")
try:
    tournaments_for_the_year_values = get_tournaments_for_year(year)
    tournament_for_year = tournaments_for_the_year_values[1]
    elapsed_time = tournaments_for_the_year_values[0]
    logger.log_info(f"executing get_tournament_for_year took {elapsed_time: .1f}s")
except Exception as e:
    logger.log_info(tournaments_for_the_year_values)
    logger.log_info(f"Impossible to retrieve data for year {year}")

In [ ]:
tournament_for_year[tournament_for_year.title.str.contains('Tianjin Open')]

In [ ]:
tournaments_for_the_year_lit = tournament_for_year
tournaments_for_the_year_lit

## II. For the tournaments  selected retrieve maths details
- Given the url of the tournament
    - retrieve all the matches (score, sets, match duration)
    - retrieve stats about the match (return points, aces, etc...)

In [ ]:
from matches.match import *
from matches.match_details import *
import unidecode
from joblib import Parallel, delayed
import time

In [ ]:
def string_normalize(string):
    return unidecode.unidecode(string).replace(" ", "_")

def create_path(*args):
    return os.path.join(*args)


## II.1 PRD-TRD
- For **prd-trd/0_raw**
    - retrieve the tournament detail
    - retrieve the matches for the tournament
    - retrieve the matches details for the tournament

In [ ]:
# verifiy if the PRD-TRD folder exist for the given year, otherwise, create it
container = "prd-trd"
level = "0_raw"
size = len(tournaments_for_the_year_lit)
#get the list of the tourney based on tournaments_for_year
for index, tourney in tournaments_for_the_year_lit.iterrows():
    print(f" donnees de tournoi brut: {len(tournaments_for_the_year_lit)}")
    print(f" donnees de tournoi deduplicated: {len(tournaments_for_the_year_lit.drop_duplicates())}")
    if index>=5:
        logger.log_info(f"processing tourney {index}/{size} {tourney['title']} for year {year}")
        data_tournament_values = retrieve_matches_html(tourney['scores_link'])
        data_tournament = data_tournament_values[1]
        elapsed_time = data_tournament_values[0]
        logger.log_info(f"executing retrieving tournament {tourney['title']} matches  took {elapsed_time: .1f}s")
        #store the data
        tourney_dir_path = os.path.join(os.getcwd(), container, level, year, string_normalize(tourney['title']))
        os.makedirs(tourney_dir_path, exist_ok=True)
        logger.log_info(f"tourney data will be stored here {tourney_dir_path}")                                
        #taking the line of the tournament
        tournaments_for_the_year_to_save = tournaments_for_the_year_lit[tournaments_for_the_year_lit['title']==tourney['title']]
        tournaments_for_the_year_to_save.to_csv(create_path(tourney_dir_path, string_normalize(f"{tourney['title']}.csv")), sep=";", index=False)
        #taking the matches of the tournament
        logger.log_info(f"data tournaments data will be stored here {tourney_dir_path}")
        print(f"tourney link: {tourney['scores_link']}")
        data_tournament["scores_link"] = tourney['scores_link']
        data_tournament.to_csv(create_path(tourney_dir_path, "matches.csv"), sep=";", index=False)
        # une autre boucle
        data_tournament_details = []
        tourney_time = 0
        print(f" donnees de match brut: {len(data_tournament)}")
        print(f" donnees de match deduplicated: {len(data_tournament.drop_duplicates())}")
        for index_match, match in data_tournament.drop_duplicates().iterrows():
            try:
                match_details_value = retrieving_match_data(match["url_match_stats"])
                match_details = match_details_value[1]
                elapsed_time = match_details_value[0]
                tourney_time += round(elapsed_time, 1)
                logger.log_info(f" elapsed time: {elapsed_time: .1f}s year: {year} tourney: {tourney['title']} index: {index_match}/{len(data_tournament.drop_duplicates())}, stage: {match['stage']}")
            except Exception as e:
                logger.log_info(f"error treating line {index_match}/{len(data_tournament.drop_duplicates())}, {tourney['title']}, url {match['url_match_stats']}")
            data_tournament_details.append(match_details)
            time.sleep(5)
        data_tournament_details_safe = [elt for elt in data_tournament_details if isinstance(elt, dict)]
        data_tournament_skipped = [elt for elt in data_tournament_details if not isinstance(elt, dict)]
        logger.log_info(f"retrieve all matches from {tourney['title']} in {tourney_time: .1f}s")
        logger.log_info(f"matchs skipped for tourney {tourney['title']}:{len(data_tournament_skipped)}")
        data_tournament_details_pd = pd.DataFrame(data_tournament_details_safe)
        #write to csv
        data_tournament_details_pd = data_tournament_details_pd.to_csv(create_path(tourney_dir_path,"matches_details.csv"), sep=";", index=False)
        print("==================================================================")
        time.sleep(60)

In [ ]:
tourney['scores_link']

In [ ]:
retrieve_matches_html(tourney['scores_link'])

## II.1 PRD-TRD
- For **prd-trd/1_quality**
    - **matches** correct the data if needed
    - **matches_details** correct the data if needed

### II.2 Iter over the **0_raw** of the year for **matches**

In [ ]:
import glob
import os
base_path = os.getcwd()
level = 'prd-trd'
sub_level = '0_raw'
regex = "*"
all_matches = glob.glob(os.path.join(base_path, level, sub_level,regex))

def matches_quality(path):
    data = pd.read_csv(path, sep=";")
    